# Step #1 Imports and Authentication

In [6]:
# A tutorial for this file is available at www.relataly.com
# Tested with Python 3.9.13, 1.2.0, Pandas 1.3.4, OpenAI 0.27.3, Tweepy 4.13.0, Requests 2.26.0, Azure.identity 1.7.0, azure.keyvault.secrets 4.6.0

import logging
import pandas as pd
import openai
import tweepy
import csv
import requests
from azure.keyvault.secrets import SecretClient
from azure.identity import AzureCliCredential

# Set API Keys and Authentification
logging.info('Setting Key Vault Auth')
client = SecretClient(f"https://{'keyvaultforbot'}.vault.azure.net/", AzureCliCredential())

logging.info('Setting NewsAPI API Key')
NEWSAPI_API_KEY = client.get_secret('newsapi-api-key').value

logging.info('Setting OpenAI API Key')
OPENAI_API_KEY = client.get_secret('openai-api-key').value
openai.api_key = OPENAI_API_KEY

logging.info('Setting Twitter API Key')
auth=tweepy.OAuthHandler(client.get_secret('twitter-api-key').value,
                         client.get_secret('twitter-api-secret').value)
auth.set_access_token(client.get_secret('twitter-access-token').value,
                      client.get_secret('twitter-access-secret').value)
twitter_api=tweepy.API(auth)

CSV_NAME = 'news_log.csv'

# Step #2 Function for News API

In [7]:
#### NewsAPI
def fetch_news(number=10):
    # Fetch tech news from NewsAPI
    url = f"https://newsapi.org/v2/top-headlines?country=us&category=technology&apiKey={NEWSAPI_API_KEY}"
    response = requests.get(url).json()
    news_items = response["articles"]
    df = pd.DataFrame(news_items)
    df = df[["title", "description", "url"]].dropna()
    return df.head(number)

# Step #3 OpenAI Functions for News Relevance and Tweet Generation

In [8]:
#### OpenAI Engine
def openai_request(instructions, task, model_engine='gpt-3.5-turbo'):
    prompt = [{"role": "system", "content": instructions }, 
              {"role": "user", "content": task }]
    completion = openai.ChatCompletion.create(model=model_engine, messages=prompt, temperature=0.5, max_tokens=300)
    return completion.choices[0].message.content


#### Define OpenAI Prompt for news Relevance
def select_relevant_news_prompt(news_articles, topic):    
    instructions = f'Your job is to check news articles for their relevance on a given topic.'
    task =  f"check if the following news_articles are relevant to a topic from the following list: {topic}. \
        News: {news_articles}. \
        Add the relevance in terms to a list with True if the article is relevant, False if not. \
        You must not return any Explanation and just return the list. \
        ## \
        Sample News: ['new AI model available from Nvidia', 'We Exploded the AMD Ryzen 7 7800X3D & Melted the Motherboard', \
        'The Lara Croft Collection For Switch Has Been Rated By The ESRB - Nintendo Life'] \
        [True, False, False]"
    return instructions, task


#### Define OpenAI Prompt for News Tweet
def create_tweet_prompt(title, description, tiny_url):
    instructions = f'You are a twitter user that creates tweets with a maximum length of 280 characters.'
    task = f"Create an informative tweet on twitter based on the following news title and description. \
        The tweet must use a maximum of 280 characters. \
        Include the {tiny_url}. But do not include any other urls.\
        Title: {title}. \
        Description: {description}. \
        Use hashtags to reach a wider audience. \
        Do not include any emojis in the tweet"
    return instructions, task

# Step #4 Functions for Tweeting

In [9]:
def check_tweet_length(tweet):
    if len(tweet) > 280:
        print(f'Tweet too long: {len(tweet)}')
        return False
    else:
        print(f'Tweet length OK: {len(tweet)}')
        return True


def create_tweet(title, description, url):
    # create tiny url
    tiny_url = create_tiny_url(url)

    # define prompt for tweet creation
    instructions, task = create_tweet_prompt(title, description, tiny_url)
    tweet = openai_request(instructions, task)
    tweet = tweet.replace('"', '')

    # check tweet length and post tweet
    if check_tweet_length(tweet):
            print(f'Creating tweet: {tweet}')
            status = twitter_api.update_status(tweet)
            logging.info(f'Tweet posted: {status.id}')
    else: 
        status = 'error tweet too long'
    return status


def create_tiny_url(url):
    response = requests.get(f'http://tinyurl.com/api-create.php?url={url}')
    return response.text

# Step #5 Bringing it All Together - Running the Bot

In [10]:
def main_bot():
    # Read the old CSV data
    # try opening the csv file and creeate it if it does not exist
    try:
        df_old = pd.read_csv(CSV_NAME)
    except:
        # create the csv file
        df_old = pd.DataFrame(columns=['title'])
        df_old.to_csv(CSV_NAME, index=False)

    # Fetch news data
    df = fetch_news(10)

    # Check the Relevance of the News and Filter those not relevant
    relevant_topics ='[AI, Python Programming, Machine Learning, Applications of Artificial Intelligence, Software Engineering]'
    instructions, task = select_relevant_news_prompt(list(df['title']), relevant_topics)
    relevance = openai_request(instructions, task)
    relevance = eval(relevance)

    # Filter the news data based on OpenAI response
    df['relevance'] = relevance
    display(df)

    df = df[relevance]

    if len(df) > 0:
        for i, row in df.iterrows():
            title = row['title']
            description = row['description']
            url = row['url']
            
            # Check if the title has already been tweeted
            if title not in df_old.title.values:
                # Create a tweet
                response = create_tweet(title, description, url)
                print(response)

                print(f"Tweeted: {title}")
                with open(f'{CSV_NAME}', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([title])
                
            else: 
                print(f"Already tweeted: {title}")
                logging.info(f"Already tweeted: {title}")
    else: 
        print("No news articles found")
        logging.info("No news articles found")

main_bot()

,title,description,url,relevance
0,6 Things Horizon 3 Needs to Succeed - Beyond C...,"In this clip of Podcast Beyond, Jada Griffin, ...",https://www.youtube.com/watch?v=2eHTGrobyEg,False
1,MMORPG Chrono Odyssey 'Gameplay Reveal' traile...,South Korea-based game developer NPIXEL has re...,https://www.gematsu.com/2023/05/mmorpg-chrono-...,False
2,Google is replacing Chrome’s misleading lock i...,Google announced that the lock will be replace...,https://www.theverge.com/2023/5/3/23709498/goo...,False
3,The new Pokémon Trading Card Game app is final...,Pokémon Trading Card Game Live will launch on ...,https://www.theverge.com/2023/5/3/23709584/pok...,False
4,Apple and Google are teaming up on a plan to m...,Apple and Google are working together on a new...,https://www.cnn.com/2023/05/03/tech/airtags-ap...,False
5,Researchers See Through a Mouse's Eyes by Deco...,"Don't worry, the technology can't be used to r...",https://gizmodo.com/ai-deep-learning-brain-act...,True
6,F1 23 release date announced with Braking Poin...,"F1 2023 launches June 16, 2023 for PlayStation...",https://www.polygon.com/23709519/f1-23-game-re...,False
7,iOS 17 Coming Soon for iPhones and Rumored to ...,Apple is expected to announce iOS 17 during it...,https://www.macrumors.com/2023/05/03/ios-17-ru...,False
8,Pokémon GO's Monthly Earnings Have Fallen To A...,"Only $34.7 million, the horror",https://www.nintendolife.com/news/2023/05/poke...,False
9,Double Dragon Gaiden: Rise of the Dragons - Of...,Watch the announcement trailer for Double Drag...,https://www.youtube.com/watch?v=MksA6p7rqOM,False


Already tweeted: Researchers See Through a Mouse's Eyes by Decoding Brain Signals - Gizmodo
